In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
from configobj import ConfigObj
from glob import glob

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'
# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = '243'
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '243':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [32]:
def len_of_RC_time(patientunitstayid):
    query = query_schema + """
    select *
    from respiratorycharting
    where patientunitstayid = '{}'
    order by respchartoffset
    """.format(patientunitstayid)

    df_respiratorycharting = pd.read_sql_query(query, con)
    df_respiratorycharting = df_respiratorycharting[['patientunitstayid','respchartoffset']]
    if df_respiratorycharting.size == 0:
        RC_time = 0
    else:
        RC_time = df_respiratorycharting['respchartoffset'][len(df_respiratorycharting.index)-1] - df_respiratorycharting['respchartoffset'][0]
    return RC_time

In [7]:
#Patient
start = time.clock()
#uniquepid = '025-19271'
query = query_schema + """
select *
from patient,treatment
where treatmentstring like '%dialysis%' and patient.patientunitstayid=treatment.patientunitstayid and activeupondischarge='False'
order by uniquepid,patient.patientunitstayid
"""

df_patient = pd.read_sql_query(query, con)
df_patient = df_patient[['uniquepid','patienthealthsystemstayid','patientunitstayid','unitvisitnumber','gender','age','admissionheight','admissionweight', 'dischargeweight',
                         'hospitaladmitoffset', 'hospitaldischargeoffset', 'hospitaladmittime24', 'hospitaldischargetime24']]
#df_patient.to_excel(os.getcwd()+'\\patient.xlsx',index=False,sheet_name='patient')

df = pd.DataFrame(df_patient)

for i in range (0,len(df.columns)):
    if df.columns[i] == 'admissionheight':
        tempC = list(df.columns)
        tempC[i] = 'height'
        df.columns = tempC
    elif df.columns[i] == 'admissionweight':
        tempC = list(df.columns)
        tempC[i] = 'weight'
        df.columns = tempC
for i in range(0,len(df.index)):  
    if pd.isnull(df['weight'][i]):
        df['weight'][i] = df['dischargeweight'][i]
for i in range(0,len(df.columns)):
    if df.columns[i] == 'dischargeweight':
        break
droplist = []
for j in range(i,len(df.columns)):  
    droplist.append(df.columns[j])
df = df.drop(droplist,axis=1)
#df.to_csv(os.getcwd()+'\\patientonly.csv',index=False)
#計算總住院天數及第一天換日分鐘數
space_columns = np.zeros((len(df.index),), dtype=int)
a=[]
b=[]
for i in range(0,len(df_patient.index)):
    struct_time = time.strptime(df_patient['hospitaladmittime24'][i],'%H:%M:%S')
    hospitaladmittime = struct_time.tm_hour*60 + struct_time.tm_min + struct_time.tm_sec/60
    Firstday = df_patient['hospitaladmitoffset'][i] + (24*60 - hospitaladmittime)
    lenofstay = (df_patient['hospitaldischargeoffset'][i] - Firstday)/ (24*60) + 1
    if (lenofstay - int(lenofstay)) > 0:
        lenofstay = lenofstay + 1
    print(int(lenofstay), Firstday)
    a.append(int(lenofstay))
    b.append(Firstday)
    print("done")
df.insert(len(df.columns),"LoS",a)
df.insert(len(df.columns),"Firstday",b)
df.to_csv(os.getcwd()+'\\allpatient1012treatment.csv',index=False)
end = time.clock() - start
print(end)

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
18 132.0
done
3 149.0
done
3 149.0
done
3 149.0
done
3 149.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 -3112.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
9 70.0
done
4 439.0
done
4 439.0
done
4 439.0
done
4 439.0
done
4 439.0
done
4 439.0
done
4 553.0
done
4 553.0
done
7 -4052.0
done
3 1327.0
done
3 1327.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
17 284.0
done
6 -493.0

9 69.0
done
9 69.0
done
9 69.0
done
9 69.0
done
4 918.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
9 95.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
7 879.0
done
14 -2562.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 243.0
done
7 126.0
done
7 126.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 -3764.0
done
12 144.0
done
12 144.0
done
12 144.0
done
12 144.0
done
12 144.0
done
12 144.0
done
12 144.0

17 -5721.0
done
17 -5721.0
done
32 -34229.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
32 -483.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
11 924.0
done
13 -4919.0
done
13 -4919.0
done
13 -4919.0
done
8 -711.0
done
8 -711.0
done
8 -711.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
9 -88.0
done
2 464.0
done
2 464.0
done
6 164.0
done
13 -8538.0
done
13 -8538.0
done
13 -8538.0
done
13 -8538.0
done
13 -8538.0
done
13 -8538.0
done
13 -8538.0
done


16 -11610.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
11 263.0
done
12 87.0
done
12 87.0
done
12 87.0
done
12 87.0
done
2 77.0
done
2 77.0
done
2 77.0
done
10 974.0
done
10 974.0
done
10 974.0
done
10 974.0
done
10 974.0
done
10 974.0
done
32 -40313.0
done
32 -40313.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
19 -9232.0
done
5 -3978.0
done
5 -3978.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
16 421.0
done
5 1335.0
done
5 1335.0
done
13 -936.0
done
23 -152.0
done
23 -152.0
done
23 -152.0
done
23 -152.0
done
23 -152.0
done
23 -152.0
done
12 -5456.0
done
12 -5456.0
done
12 -5456

...

39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
39 -8690.683333333334
done
108 -29895.0
done
108 -29895.0
done
108 1221.0
done
108 1221.0
done
108 1221.0
done
108 -121907.0
done
62 197.0
done
62 197.0
done
62 197.0
done
62 197.0
done
62 197.0
done
31 -2055.0
done
31 -2055.0
done
31 -2055.0
done
31 -2055.0
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
46 -10005.166666666666
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
12 -2387.0
done
1

26 -44.700000000000045
done
26 -44.700000000000045
done
26 -44.700000000000045
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
6 290.0
done
2 218.5
done
2 218.5
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
10 1288.8333333333333
done
107 3.0
done
107 3.0
done
107 3.0
done
107 3.0
done
107 3.0
done
107 3.0
done
107 3.0
done
107 3.0
done
5 31.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
16 486.0
done
5 -877.0
done
5 -877.0
done
5 -877.0
done
51 -60710.0
done
51 -60710.0
done
51 -60710.0
done
51 -313.0
done
51 -313.0
done
51 -313.0
done
51 -313.0
done
51 -313.0
done
51 -313.0
do

54 -57782.0
done
54 -57782.0
done
54 -57782.0
done
54 -57782.0
done
54 -57782.0
done
54 -57782.0
done
54 -28865.0
done
54 -28865.0
done
18 -5341.0
done
18 -5341.0
done
18 -5341.0
done
92 562.0
done
92 562.0
done
92 562.0
done
92 562.0
done
92 562.0
done
92 562.0
done
92 562.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
97 -4477.0
done
14 -1382.0
done
14 -1382.0
done
14 -1382.0
don

203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
203 533.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
16 416.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
28 398.0
done
5 1293.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
24 -1216.0
done
36 170.0
done
36 170.0
done
36 170.0
done
36 170.0
done


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:56: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [11]:
df=pd.read_csv(os.getcwd()+'\\allpatient1012treatment.csv')
df=df.drop_duplicates()
df.to_csv(os.getcwd()+'\\allpatient1012treatment.csv',index=False)

In [20]:
df=pd.read_csv(os.getcwd()+'\\allpatient.csv')#去除重複
for i in range (1,len(df)):
    df.drop(df.loc[df.patientunitstayid== df.patientunitstayid.shift()].index, inplace=True)
df.to_csv(os.getcwd()+'\\allpatientnoduplicate.csv',index=False)

In [6]:
df=pd.read_csv(os.getcwd()+'\\allpatient20220101filter.csv')
df2=pd.read_csv(os.getcwd()+'\\vitalperiodic_pid_drop.csv')
droplist=[]
for i in range(0,len(df2)):
    for j in range(0,len(df)):
        if(df['patientunitstayid'][j]==df2['0'][i]):
            droplist.append(j)
df=df.drop(droplist)
print(droplist)
df.to_csv(os.getcwd()+'\\allpatient20220104filter.csv',index=False)

[15, 23, 82, 29, 80, 50, 46, 47, 48, 112, 90, 66, 74, 94, 42, 113, 62, 63, 64, 0, 4, 75, 86, 43, 104, 101, 111, 10, 35, 65, 81, 39, 72, 102, 77, 105, 19, 20, 2, 93, 45, 58, 8, 11, 12, 13, 14, 56, 31, 32, 61, 100, 131, 289, 141, 158, 175, 186, 178, 168, 118, 183, 145, 223, 147, 247, 222, 189, 254, 204, 253, 180, 265, 139, 286, 287, 205, 246, 281, 198, 221, 207, 218, 176, 262, 237, 335, 438, 389, 328, 429, 456, 301, 298, 347, 451, 386, 314, 623, 795, 554, 853, 494, 793, 822, 541, 944, 485, 593, 582, 687, 643, 902, 659, 952, 737, 857, 790, 564, 704, 778, 933, 738, 855, 942, 885, 614, 654, 883, 936, 869, 496, 521, 552, 697, 831, 884, 757, 741, 752, 543, 482, 608, 1001, 1005, 1074, 1043, 1004, 1021, 1085, 1020, 983, 998, 1099, 1095, 1023, 982, 1096, 1016, 1015, 1100, 1086, 1051, 1062, 1056, 1025, 1045, 1065, 1048, 1049, 1012, 1057, 1006, 1070, 1078, 994, 1124, 1175, 1122, 1135, 1179, 1194, 1311, 1292, 1270, 1262, 1240, 1303, 1223, 1279, 1231, 1316, 1397, 1369, 1361, 1427, 1428, 1426, 1423, 

In [12]:
df=pd.read_csv(os.getcwd()+'\\allpatient1012treatment.csv')
d=[]#存是否有期間內IO紀錄的boolean值

for i in range(0,len(df)):
    query = query_schema + """
    select actualicumortality
    from apachepatientresult
    where patientunitstayid = '{}' 
    """.format(df['patientunitstayid'][i])
    df_mortality=pd.read_sql_query(query, con)
    df_mortality=df_mortality[['actualicumortality']]
    try:
        if(df_mortality['actualicumortality'][0]=='ALIVE'):
            d.append(0)
        elif(df_mortality['actualicumortality'][0]=='EXPIRED'):
            d.append(1)
    except IndexError:
        d.append("None")
    #print(df_mortality['actualicumortality'][0])
df.insert(len(df.columns),"mortality",d)
df.to_csv(os.getcwd()+'\\allpatient1012treatment.csv',index=False)

In [19]:
df=pd.read_csv(os.getcwd()+'\\allpatient1012.csv')
d=[]#存是否有期間內IO紀錄的boolean值

for i in range(0,len(df)):
    query = query_schema + """
    select dialysis
    from apacheapsvar
    where patientunitstayid = '{}' 
    """.format(df['patientunitstayid'][i])
    df_dialysis=pd.read_sql_query(query, con)
    df_dialysis=df_dialysis[['dialysis']]
    try:
        d.append(df_dialysis['dialysis'][0])
    except IndexError:
        d.append("None")
    #print(df_mortality['actualicumortality'][0])
df.insert(len(df.columns),"dialysis",d)
df.to_csv(os.getcwd()+'\\allpatient1012.csv',index=False)

In [28]:
df1 = pd.read_csv(os.getcwd()+'\\intakeoutput1012_drop.csv')
df2 = pd.read_csv(os.getcwd()+'\\lab1012_dropid.csv')
df3 = pd.read_csv(os.getcwd()+'\\respiratorycharting_1012drop.csv')
#df4 = pd.read_csv(os.getcwd()+'\\vitalperiodic1007_dropid.csv')
df5 = pd.read_csv(os.getcwd()+'\\diagnosis20211012_mask.csv')
pid=[]
droplist=[]
for i in range(0,len(df1)):
    pid.append(df1['0'][i])
for i in range(0,len(df2)):
    pid.append(df2['0'][i])
for i in range(0,len(df3)):
    pid.append(df3['0'][i])
pid = np.unique(pid).tolist()
print(len(pid))
for i in range(0,len(pid)):
    for j in range(0,len(df5)):
        if(pid[i]==df5['patientunitstayid'][j]):
            droplist.append(j)
            continue
df5=df5.drop(droplist)
df5.index = range(0,len(df5.index))
df5.to_csv('diagnosis20211012_mask.csv',index=False)

1063


In [31]:
a=[1,2,0.001,3,4,0.001]
a=list(filter((0.001).__ne__, a))
print(a)

[1, 2, 3, 4]


In [8]:
df=pd.read_csv(os.getcwd()+'\\allpatient1012drop.csv')
novitalpatient=[]
for i in range(0,len(df)):
    query = query_schema + """
    select *
    from vitalperiodic
    where patientunitstayid = '{}' 
    """.format(df['patientunitstayid'][i])
    df_vital=pd.read_sql_query(query, con)
    if(len(df_vital)==0):
        novitalpatient.append(df['patientunitstayid'][i])
novitalpatient.to_csv(os.getcwd()+'\\patientnovital.csv',index=False)

KeyboardInterrupt: 